In [339]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()
gezi.set_pandas()

In [340]:
V = 7
root = f'../working/offline/{V}/0'
# pairwise two tower, recall model
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_ext_emlm_mlm.ep1'
# p2t_model = 'all-mpnet-base-v2.flag-pairwise14-2'
pt_model = 'all-mpnet-base-v2.flag-pairwise14-2-pre_mlm3'
# pt_model = 'all-mpnet-base-v2.flag-pairwise14-2'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_emlm_mlm-mmnilm'
# pt_model2 = 'pmminilm.flag-pairwise14-2-pre_mlm3-pmminilm'
pt_model2 = 'pmminilm.flag-pairwise14-2'
# pairwise concat, rank model
# pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-extpred-ft.neg_rand_prob-0.neg_strategy-rand-sample'
pc_model = 'deberta-v3-small.flag-pairwise14-4-cat-insert-oof-ft'

# context model
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model2 = 'lsg-mminilm.flag-context4-3-d-s-mminilm'
# c_model = 'deberta-v3-small.flag-context4-2-d'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft'
c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
# c_model = 'deberta-v3-small.flag-context4-2-d-extpred-ft.0804'
# c_model = 'deberta-v3-small.flag-context4-2-d-oof-ft.list_train_ordered'
c_model2 = 'lsg-mminilm.flag-context4-3-d-s-oof-ft-mminilm'

In [341]:
df_train = pd.read_feather('../working/train.fea')
df_train = df_train[df_train.fold==0]

In [342]:
xp = gezi.load(f'{root}/{pt_model}.eval/valid.pkl')

In [343]:
xp.keys()

dict_keys(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred', 'n_words', 'match_rank', 'match_code', 'n_code_cell', 'n_cell', 'probs', 'sims'])

In [344]:
xp2 = gezi.load(f'{root}/{pt_model2}.eval/valid.pkl')

In [345]:
xc = gezi.load(f'{root}/{c_model}.eval/valid.pkl')

In [346]:
xpc = gezi.load(f'{root}/{pc_model}.eval/valid.pkl')

In [347]:
# xc2 = gezi.load(f'{root}/{c_model2}.eval/valid.pkl')

In [348]:
len(set(xp['id']))

27590

In [349]:
gezi.save(xp, '../working/xs/xp.pkl')
gezi.save(xp2, '../working/xs/xp2.pkl')
gezi.save(xc, '../working/xs/xc.pkl')
gezi.save(xpc, '../working/xs/xpc.pkl')
# gezi.save(xc2, '../working/xs/xc2.pkl')

In [350]:
xpc.keys()

dict_keys(['id', 'cell_id', 'cid', 'cell_type', 'rank', 'rel_rank', 'match_code', 'n_code_cell', 'n_markdown_cell', 'n_cell', 'code_idxes', 'n_words', 'pred', 'cls_pred_ori', 'cls_pred', 'probs', 'match_rank'])

In [351]:
# calc_metric(xc, 'reg_pred')

In [352]:
xc.keys()

dict_keys(['pred', 'cls_pred', 'cid', 'group_pos', 'group_rank', 'pred_id', 'id', 'cell_id', 'cell_type', 'rank', 'rel_rank', 'n_code_cell', 'n_markdown_cell', 'n_cell', 'n_words', 'reg_pred', 'cls_pred_ori', 'cls2_pred'])

In [353]:
# d = pd.DataFrame(gezi.batch2list(xc))
# dg = d.groupby('pred_id')
# max_preds = dg['pred'].max()
# min_preds = dg['pred'].min()
# max_cls_preds = dg['cls_pred'].max()
# min_cls_preds = dg['cls_pred'].min()

In [354]:
gezi.sort_dict_byid_(xp, 'cid')
gezi.sort_dict_byid_(xp2, 'cid')
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xpc, 'cid')
# gezi.sort_dict_byid_(xc2, 'cid')

In [355]:
num_negs = xpc['cls_pred_ori'].shape[1] 
pc_preds = dict(zip(xpc['cid'], xpc['pred']))
pc_cls_preds = dict(zip(xpc['cid'], xpc['cls_pred']))
pc_probs = dict(zip(xpc['cid'], xpc['probs']))

preds = []
cls_preds = []
probs = []
marks = []
for cid, pred, cls_pred, prob in zip(xp['cid'], xp['pred'], xp['cls_pred'], xp['probs']):
  preds.append(pc_preds.get(cid, pred))
  cls_preds.append(pc_cls_preds.get(cid, cls_pred))
  if prob.max() < 0.4:
    marks.append(1)
    probs.append(pc_probs.get(cid, prob))
  else:
    probs.append(prob)
    marks.append(0)

xpc = xp.copy()
xpc['pred'] = np.asarray(preds)
xpc['cls_pred'] = np.asarray(cls_preds)
xpc['probs'] = np.asarray(probs)
xpc['mark'] = np.asarray(marks)
ic(xpc['mark'].mean())

[08/06/22 21:57:55] 238809541.py:25 in <module>
                    xpc['mark'].mean(): 0.13489809221472057


0.13489809221472057

In [356]:
LABEL_COL = 'rel_rank'
def diff_feats(x, l,r, name):
  if isinstance(l, str):
    x[f'{name}_diff'] = x[l] - x[r]
  else:
    x[f'{name}_diff'] = l - r
  x[f'abs_{name}_diff'] = abs(x[f'{name}_diff'])

def gen_pfeat(x_, topn=10):
  x = {}
  keys = ['pred', 'cls_pred']
  x = {k: x_[k] for k in keys}
  x['rank_pred'] = x['pred'] * (1 + x_['n_code_cell']) - 0.5
  x['cls_rank_pred'] = x['cls_pred'] * (1 + x_['n_code_cell']) - 0.5
  x['min_prob'] = x_['probs'].min()
  x['var_prob'] = np.var(x_['probs'])
  if 'sims' in x_:
    x['min_sim'] = x_['sims'].min()
    x['var_sim'] = np.var(x_['sims'])
  
  idxes = (-x_['probs']).argsort()
  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / (x_['n_code_cell'] + 1)
      x[f'top_prob_{i}'] = x_['probs'][idxes[i]]
      if 'sims' in x_:
        x[f'top_sim_{i}'] = x_['sims'][idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_prob_{i}'] = -1
      if 'sims' in x_:
        x[f'top_sim_{i}'] = -1
  
  diff_feats(x, 'cls_pred', 'pred', 'cls')
  return x

def gen_cfeat(x_):
  topn = 4
  x = {}
  keys = [
    'pred', 
    'cls_pred', 
    'reg_pred', 
    'cls2_pred', 
    'group_pos',
    'group_rank',
    ]
  x = {k: x_[k] for k in keys}
  x['rank_pred'] = x['pred'] * FLAGS.num_classes - 0.5
  x['cls_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  x['cls2_rank_pred'] = x['cls2_pred'] * FLAGS.num_classes - 0.5
  x['reg_rank_pred'] = x['cls_pred'] * FLAGS.num_classes - 0.5
  preds = x_['cls_pred_ori']
  probs = gezi.softmax(preds)
  x['min_prob'] = probs.min()
  x['var_prob'] = np.var(probs)
  idxes = (-probs).argsort()

  for i in range(topn):
    if i < len(idxes):
      if i > 0:
        x[f'top_pred_{i}'] = (idxes[i] + 0.5) / FLAGS.num_classes
      x[f'top_prob_{i}'] = probs[idxes[i]]
    else:
      x[f'top_pred_{i}'] = -1
      x[f'top_prob_{i}'] = -1
  diff_feats(x, 'cls_pred', 'reg_pred', 'cls_reg')
  diff_feats(x, 'cls_pred', 'cls2_pred', 'cls_cls2')
  diff_feats(x, 'cls2_pred', 'reg_pred', 'cls2_reg')
  
  # group_id = x_['pred_id']
  # x['max_pred'] = max_preds[group_id]
  # x['min_pred'] = min_preds[group_id]
  # x['pred_ratio'] = (x['pred'] - x['min_pred']) / (x['max_pred'] - x['min_pred']) if (x['max_pred'] - x['min_pred']) else 1
  # x['max_cls_pred'] = max_cls_preds[group_id]
  # x['min_cls_pred'] = min_cls_preds[group_id]
  # x['cls_pred_ratio'] = (x['cls_pred'] - x['min_cls_pred']) / (x['max_cls_pred'] - x['min_cls_pred']) if (x['max_cls_pred'] - x['min_cls_pred']) else 1
  
  return x

def gen_feats():
  xs = gezi.batch2list(xp)
  p_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp')]
  xs = gezi.batch2list(xp2)
  p2_feats = [gen_pfeat(x) for x in tqdm(xs, desc=f'gen_pfeats:xp2')]
  xs = gezi.batch2list(xc)
  c_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc')]
  xs = gezi.batch2list(xpc)
  pc_feats = [gen_pfeat(x, topn=5) for x in tqdm(xs, desc=f'gen_pfeats:xpc')]
  # xs = gezi.batch2list(xc2)
  # c2_feats = [gen_cfeat(x) for x in tqdm(xs, desc=f'gen_cfeats:xc2')]
  feats = []
  for i in tqdm(range(len(xs)), desc='merge_feats'):
    fe1 = p_feats[i]
    fe2= p2_feats[i]
    fec = c_feats[i]
    fepc = pc_feats[i]
    fepc['mark'] = xpc['mark'][i]
    # fec2 = c2_feats[i]
    
    fe = {}
    fe['code_ratio'] = xp['n_code_cell'][i] / xp['n_cell'][i]
    diff_feats(fe, fe1['pred'], fe2['pred'], 'ps')
    diff_feats(fe, fe1['cls_pred'], fe2['cls_pred'], 'ps_cls')
    diff_feats(fe, fe1['pred'], fec['reg_pred'], 'pc_reg')
    diff_feats(fe, fe2['pred'], fec['reg_pred'], 'p2c_reg')
    diff_feats(fe, fe1['pred'], fec['cls_pred'], 'pc_cls')
    diff_feats(fe, fe2['pred'], fec['cls_pred'], 'p2c_cls')
    diff_feats(fe, fe1['pred'], fec['cls2_pred'], 'pc_cls2')
    diff_feats(fe, fe2['pred'], fec['cls2_pred'], 'p2c_cls2')
    diff_feats(fe, fe1['cls_pred'], fec['reg_pred'], 'pc_cls_reg')
    diff_feats(fe, fe2['cls_pred'], fec['reg_pred'], 'p2c_cls_reg')
    diff_feats(fe, fe1['cls_pred'], fec['cls_pred'], 'pc_cls_cls')
    diff_feats(fe, fe2['cls_pred'], fec['cls_pred'], 'p2c_cls_cls')
    diff_feats(fe, fe1['cls_pred'], fec['cls2_pred'], 'pc_cls_cls2')
    diff_feats(fe, fe2['cls_pred'], fec['cls2_pred'], 'p2c_cls_cls2')
    
    diff_feats(fe, fepc['pred'], fec['reg_pred'], 'pcc_reg')
    diff_feats(fe, fepc['pred'], fec['cls_pred'], 'pcc_cls')
    diff_feats(fe, fepc['pred'], fec['cls2_pred'], 'pcc_cls2')
    diff_feats(fe, fepc['cls_pred'], fec['reg_pred'], 'pcc_cls_reg')
    diff_feats(fe, fepc['cls_pred'], fec['cls_pred'], 'pcc_cls_cls')
    diff_feats(fe, fepc['cls_pred'], fec['cls2_pred'], 'pcc_cls_cls2')

    fe1 = gezi.dict_prefix(fe1, 'xp_')
    fe.update(fe1)
    fe2 = gezi.dict_prefix(fe2, 'xp2_')
    fe.update(fe2)
    fec = gezi.dict_prefix(fec, 'xc_')
    fe.update(fec)
    fepc = gezi.dict_prefix(fepc, 'xpc_')
    fe.update(fepc)
    # fec2 = gezi.dict_prefix(fec, 'xc2_')
    # fe.update(fec2)
    
    keys = [
     'id', 'cell_id', 'cid',
     'n_words', 'n_code_cell', 'n_cell'
    ]
    for key in keys:
      fe[key] = xp[key][i]
      
    #  ic(feat)
    feats.append(fe)
  #  break
  dfeats = pd.DataFrame(feats)
  return dfeats

In [357]:
dfeats = gen_feats()

gen_pfeats:xp:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xp2:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_cfeats:xc:   0%|          | 0/424943 [00:00<?, ?it/s]

gen_pfeats:xpc:   0%|          | 0/424943 [00:00<?, ?it/s]

merge_feats:   0%|          | 0/424943 [00:00<?, ?it/s]

In [358]:
df = dfeats.merge(df_train[['cid', 'ancestor_id', LABEL_COL]], on='cid')

In [359]:
gezi.set_fold(df, 5, 'ancestor_id')

In [360]:
reg_cols = [x for x in dfeats.columns if x not in ['id', 'cell_id', 'cid', LABEL_COL]]
cat_cols = []
cols = reg_cols + cat_cols
len(cols)

172

In [361]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [362]:
X_train = dtrain[cols]
y_train = dtrain[LABEL_COL]
X_valid = dvalid[cols]
y_valid = dvalid[LABEL_COL]

In [363]:
from catboost import CatBoostRegressor, CatBoostRanker
cbt_params = {
              # 'bootstrap_type': 'Poisson',
              # 'learning_rate': 0.02,
              'learning_rate': 0.03,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              # 'bagging_temperature': 0.2,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              'min_child_samples': 100,
              # 'task_type': 'GPU',
              # 'devices': '0',
              }

In [364]:
cbt_model = CatBoostRegressor(**cbt_params)
cbt_model.fit(X_train, y_train,
      eval_set=[(X_train, y_train), (X_valid, y_valid)],
              verbose=100,
              )  
dvalid['cb_pred'] = cbt_model.predict(dvalid[cols])
cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})
ic(cbt_score)

0:	learn: 0.2521270	test: 0.2521270	test1: 0.2520677	best: 0.2520677 (0)	total: 117ms	remaining: 19m 30s
100:	learn: 0.0658651	test: 0.0658651	test1: 0.0677811	best: 0.0677811 (100)	total: 10.9s	remaining: 17m 51s
200:	learn: 0.0588522	test: 0.0588522	test1: 0.0612441	best: 0.0612441 (200)	total: 20.9s	remaining: 17m
300:	learn: 0.0572635	test: 0.0572635	test1: 0.0600526	best: 0.0600526 (300)	total: 29s	remaining: 15m 35s
400:	learn: 0.0562642	test: 0.0562642	test1: 0.0594592	best: 0.0594592 (400)	total: 38.1s	remaining: 15m 10s
500:	learn: 0.0555671	test: 0.0555671	test1: 0.0591370	best: 0.0591370 (500)	total: 48.8s	remaining: 15m 25s
600:	learn: 0.0550424	test: 0.0550424	test1: 0.0589438	best: 0.0589422 (599)	total: 1m	remaining: 15m 44s
700:	learn: 0.0545422	test: 0.0545422	test1: 0.0588120	best: 0.0588120 (700)	total: 1m 13s	remaining: 16m 18s
800:	learn: 0.0541464	test: 0.0541464	test1: 0.0587282	best: 0.0587281 (799)	total: 1m 30s	remaining: 17m 14s
900:	learn: 0.0537275	test: 0.

[08/06/22 22:08:09] 3249242939.py:8 in <module>
                    cbt_score: 0.9180987808225505


0.9180987808225505

In [365]:
gezi.plot.feature_importance(cbt_model, topn=20)

In [366]:
f = gezi.plot.feature_importance_df(cbt_model)

In [367]:
f.head()

,feat_name,importanace
0,xpc_cls_pred,10.1967
1,xpc_pred,6.8090
2,xp_cls_pred,5.7471
3,xp_pred,4.3232
4,xc_reg_pred,4.1838


In [368]:
f[f.feat_name=='c_cls_pred']

,feat_name,importanace


In [369]:
f[f.feat_name=='c_reg_pred']

,feat_name,importanace


In [370]:
f[f.feat_name=='c_pos']

,feat_name,importanace


In [371]:
f[f.feat_name=='c_max_pred']

,feat_name,importanace


In [372]:
import lightgbm as lgb
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 32,
          # 'num_leaves': 16,
          'min_data_in_leaf': 300,
          # 'min_data_in_leaf': 500,
          # 'max_depth': 5,
          'learning_rate': 0.03,
          # "feature_fraction": 0.8,
          # "bagging_fraction": 0.75,
          # "feature_fraction": 0.9,
          # "bagging_fraction": 0.8,
          'feature_fraction': 0.7,
          'bagging_fraction': 0.7,
          'min_data_in_bin':100,
          "lambda_l1": 1e-7,
          'lambda_l2': 1e-7,
          "random_state": 1024,
          "num_threads": 12,
          }

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

lgb_model = lgb.train(params,
                d_train,
                10000,
                valid_sets=[d_train, d_valid],
                verbose_eval=100,
                early_stopping_rounds=100)
dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
ic(lgb_score)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076466 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42239
[LightGBM] [Info] Number of data points in the train set: 342411, number of used features: 172
[LightGBM] [Info] Start training from score 0.455056
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 0.0679607	valid_1's l1: 0.0693578
[200]	training's l1: 0.0601533	valid_1's l1: 0.0616897
[300]	training's l1: 0.0586329	valid_1's l1: 0.060265
[400]	training's l1: 0.0575984	valid_1's l1: 0.059332
[500]	training's l1: 0.0571358	valid_1's l1: 0.0589539
[600]	training's l1: 0.0568726	valid_1's l1: 0.0587605
[700]	training's l1: 0.0567056	valid_1's l1: 0.0586502
[800]	training's l1: 0.056569	valid_1's l1: 0.0585723
[900]	training's l1: 0.0564228	valid_1's l1: 0.0584939
[1000]	training's l1: 0.056297	valid_1's l1: 0.0584283
[1100]	training's l1: 0.056199	valid_1's l1

[08/06/22 22:10:19] 4120804186.py:36 in <module>
                    lgb_score: 0.9189092236293597


0.9189092236293597

In [373]:
gezi.plot.feature_importance(lgb_model, topn=20)

In [374]:
f.feat_name

0               xpc_cls_pred
1                   xpc_pred
2                xp_cls_pred
3                    xp_pred
4                xc_reg_pred
5               xc_rank_pred
6           xpc_abs_cls_diff
7                    xc_pred
8                   xp2_pred
9           xc_cls_rank_pred
10         xc_cls2_rank_pred
11              xp2_cls_pred
12             xc_top_pred_3
13               xc_cls_pred
14          xc_reg_rank_pred
15              xpc_cls_diff
16            xp2_top_pred_1
17             xc_top_pred_2
18          abs_pcc_cls_diff
19            xpc_top_pred_2
20             xc_top_pred_1
21             xp_top_pred_1
22          abs_pcc_reg_diff
23         xp2_cls_rank_pred
24               xpc_min_sim
25              xc_cls2_pred
26              xp_top_sim_4
27              xp_top_sim_3
28             xp2_rank_pred
29         abs_pcc_cls2_diff
30            xpc_top_pred_1
31      abs_pcc_cls_reg_diff
32           abs_pc_cls_diff
33      xc_abs_cls_cls2_diff
34           x

In [375]:
FOLDS = 5
cbt_scores, lgb_scores = [], []
scores = []
gezi.try_mkdir('../working/trees')
for fold in tqdm(range(FOLDS)):
  dvalid = df[df.fold==fold]
  dtrain = df[df.fold!=fold]
  X_train = dtrain[cols]
  y_train = dtrain[LABEL_COL]
  X_valid = dvalid[cols]
  y_valid = dvalid[LABEL_COL]
  if fold > 0:
    cbt_model = CatBoostRegressor(**cbt_params)
    cbt_model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                  verbose=500,
                  )  
  cbt_model.save_model(f'../working/trees/{fold}.cbt')
  dvalid['cb_pred'] = cbt_model.predict(dvalid[cols])
  cbt_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values})
  ic(cbt_score)
  
  d_train = lgb.Dataset(X_train, y_train)
  d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
  if fold > 0:
    lgb_model = lgb.train(params,
                    d_train,
                    10000,
                    valid_sets=[d_train, d_valid],
                    verbose_eval=500,
                    early_stopping_rounds=100)
  lgb_model.save_model(f'../working/trees/{fold}.lgb')
  dvalid['lgb_pred'] = lgb_model.predict(dvalid[cols])
  
  lgb_score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})
  ic(lgb_score)
  score = calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': (dvalid.cb_pred.values * 0.3 + dvalid.lgb_pred.values * 0.7)})
  ic(score)
  
  cbt_scores.append(cbt_score)
  lgb_scores.append(lgb_score)
  scores.append(score)
  ic(fold, 
     np.asarray(cbt_scores).mean(), 
     np.asarray(lgb_scores).mean(),
     np.asarray(scores).mean(),
     )

  0%|          | 0/5 [00:00<?, ?it/s]

[08/06/22 22:10:30] 1460528775.py:21 in <module>
                    cbt_score: 0.9180987808225505
[08/06/22 22:10:36] 1460528775.py:36 in <module>
                    lgb_score: 0.9189092236293597
[08/06/22 22:10:38] 1460528775.py:38 in <module>
                    score: 0.9191193924869012
[08/06/22 22:10:38] 1460528775.py:43 in <module>
                    fold: 0
                    np.asarray(cbt_scores).mean(): 0.9180987808225505
                    np.asarray(lgb_scores).mean(): 0.9189092236293597
                    np.asarray(scores).mean(): 0.9191193924869012


0:	learn: 0.2521482	test: 0.2521482	test1: 0.2520588	best: 0.2520588 (0)	total: 130ms	remaining: 21m 42s
500:	learn: 0.0560802	test: 0.0560802	test1: 0.0571944	best: 0.0571944 (500)	total: 1m 4s	remaining: 20m 28s
1000:	learn: 0.0538933	test: 0.0538933	test1: 0.0567411	best: 0.0567406 (996)	total: 2m 10s	remaining: 19m 34s
1500:	learn: 0.0520550	test: 0.0520550	test1: 0.0566893	best: 0.0566829 (1353)	total: 3m 19s	remaining: 18m 46s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.05668289978
bestIteration = 1353

Shrink model to first 1354 iterations.


[08/06/22 22:14:50] 1460528775.py:21 in <module>
                    cbt_score: 0.9206339201638898


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072868 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42242
[LightGBM] [Info] Number of data points in the train set: 339696, number of used features: 172
[LightGBM] [Info] Start training from score 0.455882
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.0575369	valid_1's l1: 0.0570709
[1000]	training's l1: 0.0566904	valid_1's l1: 0.0565963
[1500]	training's l1: 0.0562107	valid_1's l1: 0.0564033
[2000]	training's l1: 0.0558452	valid_1's l1: 0.056307
[2500]	training's l1: 0.055542	valid_1's l1: 0.0562538
[3000]	training's l1: 0.0552829	valid_1's l1: 0.0562031
[3500]	training's l1: 0.0550554	valid_1's l1: 0.0561654
[4000]	training's l1: 0.0548109	valid_1's l1: 0.0561224
[4500]	training's l1: 0.0545924	valid_1's l1: 0.0560891
[5000]	training's l1: 0.0543943	valid_1's l1: 0.0560635
[5500]	training's l1: 0.0540037	v

...

[08/06/22 22:30:42] 1460528775.py:36 in <module>
                    lgb_score: 0.9239120971047396
[08/06/22 22:30:43] 1460528775.py:38 in <module>
                    score: 0.9241708857213959
[08/06/22 22:30:44] 1460528775.py:43 in <module>
                    fold: 3
                    np.asarray(cbt_scores).mean(): 0.9206497843825381
                    np.asarray(lgb_scores).mean(): 0.9213451205782868
                    np.asarray(scores).mean(): 0.921600920698977


0:	learn: 0.2521184	test: 0.2521184	test1: 0.2522320	best: 0.2522320 (0)	total: 98.5ms	remaining: 16m 24s
500:	learn: 0.0556479	test: 0.0556479	test1: 0.0594704	best: 0.0594704 (500)	total: 39.9s	remaining: 12m 36s
1000:	learn: 0.0535107	test: 0.0535107	test1: 0.0589242	best: 0.0589190 (997)	total: 1m 20s	remaining: 11m 59s
1500:	learn: 0.0516401	test: 0.0516401	test1: 0.0588782	best: 0.0588679 (1427)	total: 2m	remaining: 11m 19s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.05886792073
bestIteration = 1427

Shrink model to first 1428 iterations.


[08/06/22 22:33:24] 1460528775.py:21 in <module>
                    cbt_score: 0.9125464914102784


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42239
[LightGBM] [Info] Number of data points in the train set: 341935, number of used features: 172
[LightGBM] [Info] Start training from score 0.456522
Training until validation scores don't improve for 100 rounds
[500]	training's l1: 0.0570207	valid_1's l1: 0.059339
[1000]	training's l1: 0.0562284	valid_1's l1: 0.0588261
[1500]	training's l1: 0.0558005	valid_1's l1: 0.0586305
[2000]	training's l1: 0.0554935	valid_1's l1: 0.0585239
[2500]	training's l1: 0.0551335	valid_1's l1: 0.0584192
[3000]	training's l1: 0.0548529	valid_1's l1: 0.0583538
[3500]	training's l1: 0.0546552	valid_1's l1: 0.0583118
[4000]	training's l1: 0.0543752	valid_1's l1: 0.0582777
Early stopping, best iteration is:
[4122]	training's l1: 0.0543249	valid_1's l1: 0.0582714


[08/06/22 22:34:42] 1460528775.py:36 in <module>
                    lgb_score: 0.9129216309789512
[08/06/22 22:34:44] 1460528775.py:38 in <module>
                    score: 0.9132761849554141
[08/06/22 22:34:44] 1460528775.py:43 in <module>
                    fold: 4
                    np.asarray(cbt_scores).mean(): 0.9190291257880862
                    np.asarray(lgb_scores).mean(): 0.9196604226584195
                    np.asarray(scores).mean(): 0.9199359735502645


In [376]:
# fold: 0
#                     np.asarray(cbt_scores).mean(): 0.9181025338378637
#                     np.asarray(lgb_scores).mean(): 0.9193848140698876
#                     np.asarray(scores).mean(): 0.9195238841373282

#  fold: 4
#                     np.asarray(cbt_scores).mean(): 0.9189608630607822
#                     np.asarray(lgb_scores).mean(): 0.9196959050732708
#                     np.asarray(scores).mean(): 0.9199400102856614

In [377]:
dvalid

,code_ratio,ps_diff,abs_ps_diff,ps_cls_diff,abs_ps_cls_diff,pc_reg_diff,abs_pc_reg_diff,p2c_reg_diff,abs_p2c_reg_diff,pc_cls_diff,abs_pc_cls_diff,p2c_cls_diff,abs_p2c_cls_diff,pc_cls2_diff,abs_pc_cls2_diff,p2c_cls2_diff,abs_p2c_cls2_diff,pc_cls_reg_diff,abs_pc_cls_reg_diff,p2c_cls_reg_diff,abs_p2c_cls_reg_diff,pc_cls_cls_diff,abs_pc_cls_cls_diff,p2c_cls_cls_diff,abs_p2c_cls_cls_diff,pc_cls_cls2_diff,abs_pc_cls_cls2_diff,p2c_cls_cls2_diff,abs_p2c_cls_cls2_diff,pcc_reg_diff,abs_pcc_reg_diff,pcc_cls_diff,abs_pcc_cls_diff,pcc_cls2_diff,abs_pcc_cls2_diff,pcc_cls_reg_diff,abs_pcc_cls_reg_diff,pcc_cls_cls_diff,abs_pcc_cls_cls_diff,pcc_cls_cls2_diff,abs_pcc_cls_cls2_diff,xp_pred,xp_cls_pred,xp_rank_pred,xp_cls_rank_pred,xp_min_prob,xp_var_prob,xp_min_sim,xp_var_sim,xp_top_prob_0,xp_top_sim_0,xp_top_pred_1,xp_top_prob_1,xp_top_sim_1,xp_top_pred_2,xp_top_prob_2,xp_top_sim_2,xp_top_pred_3,xp_top_prob_3,xp_top_sim_3,xp_top_pred_4,xp_top_prob_4,xp_top_sim_4,xp_top_pred_5,xp_top_prob_5,xp_top_sim_5,xp_top_pred_6,xp_top_prob_6,xp_top_sim_6,xp_top_pred_7,xp_top_prob_7,xp_top_sim_7,xp_top_pred_8,xp_top_prob_8,xp_top_sim_8,xp_top_pred_9,xp_top_prob_9,xp_top_sim_9,xp_cls_diff,xp_abs_cls_diff,xp2_pred,xp2_cls_pred,xp2_rank_pred,xp2_cls_rank_pred,xp2_min_prob,xp2_var_prob,xp2_min_sim,xp2_var_sim,xp2_top_prob_0,xp2_top_sim_0,xp2_top_pred_1,xp2_top_prob_1,xp2_top_sim_1,xp2_top_pred_2,xp2_top_prob_2,xp2_top_sim_2,xp2_top_pred_3,xp2_top_prob_3,xp2_top_sim_3,xp2_top_pred_4,xp2_top_prob_4,xp2_top_sim_4,xp2_top_pred_5,xp2_top_prob_5,xp2_top_sim_5,xp2_top_pred_6,xp2_top_prob_6,xp2_top_sim_6,xp2_top_pred_7,xp2_top_prob_7,xp2_top_sim_7,xp2_top_pred_8,xp2_top_prob_8,xp2_top_sim_8,xp2_top_pred_9,xp2_top_prob_9,xp2_top_sim_9,xp2_cls_diff,xp2_abs_cls_diff,xc_pred,xc_cls_pred,xc_reg_pred,xc_cls2_pred,xc_group_pos,xc_group_rank,xc_rank_pred,xc_cls_rank_pred,xc_cls2_rank_pred,xc_reg_rank_pred,xc_min_prob,xc_var_prob,xc_top_prob_0,xc_top_pred_1,xc_top_prob_1,xc_top_pred_2,xc_top_prob_2,xc_top_pred_3,xc_top_prob_3,xc_cls_reg_diff,xc_abs_cls_reg_diff,xc_cls_cls2_diff,xc_abs_cls_cls2_diff,xc_cls2_reg_diff,xc_abs_cls2_reg_diff,xpc_pred,xpc_cls_pred,xpc_rank_pred,xpc_cls_rank_pred,xpc_min_prob,xpc_var_prob,xpc_min_sim,xpc_var_sim,xpc_top_prob_0,xpc_top_sim_0,xpc_top_pred_1,xpc_top_prob_1,xpc_top_sim_1,xpc_top_pred_2,xpc_top_prob_2,xpc_top_sim_2,xpc_top_pred_3,xpc_top_prob_3,xpc_top_sim_3,xpc_top_pred_4,xpc_top_prob_4,xpc_top_sim_4,xpc_cls_diff,xpc_abs_cls_diff,xpc_mark,id,cell_id,cid,n_words,n_code_cell,n_cell,ancestor_id,rel_rank,fold,cb_pred,lgb_pred
3,0.5692,-0.0263,0.0263,0.0176,0.0176,-0.0042,0.0042,0.0222,0.0222,-0.0026,0.0026,0.0237,0.0237,-0.0034,0.0034,0.0229,0.0229,-0.0041,0.0041,-0.0217,0.0217,-0.0026,0.0026,-0.0201,0.0201,-0.0034,0.0034,-0.0210,0.0210,-0.0042,0.0042,-0.0026,0.0026,-0.0034,0.0034,-0.0076,0.0076,-0.0060,0.0060,-0.0068,0.0068,0.8816,0.8816,33.0000,33.0007,0.0000,0.0256,0.1710,0.0139,0.9990,0.7477,0.9079,0.0010,0.5951,0.8289,0.0000,0.5295,0.6184,0.0000,0.5057,0.9342,0.0000,0.4822,0.7500,0.0000,0.4765,0.6711,0.0000,0.4511,0.9605,0.0000,0.4507,0.4342,0.0000,0.4434,0.5395,0.0000,0.4349,0.0000,0.0000,0.9079,0.8640,34.0000,32.3327,0.0000,0.0138,0.0410,0.0184,0.7285,0.5429,0.6184,0.1146,0.4922,0.8816,0.0592,0.4741,0.9342,0.0473,0.4680,0.6711,0.0184,0.4421,0.6974,0.0078,0.4186,0.7500,0.0065,0.4135,0.8289,0.0057,0.4103,0.6447,0.0036,0.3978,0.7763,0.0017,0.3773,-0.0439,0.0439,0.8850,0.8842,0.8857,0.8850,2,1,87.9951,87.9160,88.0000,87.9160,0.0000,0.0096,0.9844,0.8750,0.0104,0.8550,0.0022,0.9050,0.0015,-0.0016,0.0016,-0.0008,0.0008,-0.0007,0.0007,0.8816,0.8782,33.0000,32.8708,0.0000,0.0256,0.1710,0.0139,0.9990,0.7477,0.9079,0.0010,0.5951,0.8289,0.0000,0.5295,0.6184,0.0000,0.5057,0.9342,0.0000,0.4822,-0.0034,0.0034,0,000efd285fb982,14f36391,000efd285fb982\t14f36391,102,37,65,389f14ff,0.8816,4,0.8821,0.8834
4,0.5692,0.0000,0.0000,-0.0095,0.0095,-0.0028,0.0028,-0.0028,0.0028,-0.0072,0.0072,-0.0072,0.0072,-0.0061,0.0061,-0.0061,0.0061,-0.0130,0.0130,-0.0035,0.